In [22]:
import pandas as pd
import numpy as np
import pickle
from helper_functions import Helper

In [23]:
if __name__ == "__main__":
    # args
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    CA_IN_FP  = '../data/derived/ca_cm_cw.pkl'
    CG_IN_FP  = '../data/derived/cg_cm_cw.pkl'
    CMC_IN_FP = '../data/derived/cmc_cm_cw.pkl'
    SAN_IN_FP =  '../data/derived/san_cm_cw.pkl'
    OUT_FP = '../data/clean/cw.pkl'

    # import
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe = Helper.findUniqueAssets(asset_universe_dict)
    ca_cw_df = pd.read_pickle(CA_IN_FP)
    cg_cw_df = pd.read_pickle(CG_IN_FP)
    cmc_cw_df = pd.read_pickle(CMC_IN_FP)
    san_cw_df = pd.read_pickle(SAN_IN_FP)

    # fix error in cws
    san_cw_df.loc[san_cw_df.asset_cm.isin(['gmt', 'cro']), 'asset_san'] = np.nan
    cg_cw_df.loc[cg_cw_df.asset_cm.isin(['aca', 'cro', 'ava']), 'asset_cg'] = np.nan
    cg_cw_df = cg_cw_df[~cg_cw_df.asset_cg.isin(['atlas-aggregator', 'chronos-finance', 'sincronix'])]
    cmc_cw_df.loc[cmc_cw_df.asset_cm=='gmt', 'asset_cmc'] = np.nan
    cmc_cw_df = cmc_cw_df[~cmc_cw_df.asset_cmc.isin(['atlas-aggregator', 'chronos-arbitrum', 'kryptai', 
                                                    'kinance', 'nanomatic', 'neo-', 'sincronix'])]

    # subset assets to relevant universe
    

    # merge 
    ca_cw_df = ca_cw_df[ca_cw_df.asset_cm.isin(asset_universe)]
    cw_df = ca_cw_df.merge(san_cw_df, on='asset_cm', how='inner', validate='one_to_one')
    cw_df = cw_df.merge(cg_cw_df, on='asset_cm', how='inner', validate='one_to_one')
    cw_df = cw_df.merge(cmc_cw_df, on='asset_cm', how='inner', validate='one_to_one')

    # sort rows and columns
    cw_df = cw_df[['asset_cm', 'asset_ca', 'asset_san', 'asset_cmc', 'asset_cg']]
    cw_df = cw_df.sort_values(by='asset_cm', ignore_index=True)

    # drop 'ampl' from asset universe, a stablecoin
    cw_df = cw_df[cw_df.asset_cm!='ampl']

    # output
    cw_df.to_pickle(OUT_FP)